## Pulls feature from the Mark's featureful data

and try to populate the kg with those feature

NOTE:
- Training set only first.

## Feature of battery_dataset

battery_dataset : dict() with batteryID as key (ex . 'b1c1', 'b1c3', 'b1c5', 'b1c7', 'b1c11', 'b1c15', 'b1c17', 'b1c19', 'b1c21')

battery_dataset['b1c1'] : dict with 3 keys ['q_d_n', 'trimmed_q_d_n', 'cycle', 'mean_backhalf_grad', 'gradient_q_d_n']
- cycle : containing int showing the amount of cycle until consider as degraded.
- q_d_n : array of float showing the discharge capacity left? of each cycle. (unused)
- trimmed_q_d_n : array of float showing the discharge capacity left? of each cycle (length == cycle).
- mean_backhalf_grad : floating point number representing the mean of gradient from all of the backhalf element of the array in q_d_n
- gradient_q_d_n : array of float showing the grad between discharge capacity left (length == cycle).



## NODE & RELATION IDEA

NODE:

1.Battery:
    {
    - batteryID: (e.g., "b1c1").
    - mean_backhalf_grad (mean gradient from back-half elements of trimmed_q_d_n).
    - degradation_cycle (from cycle).
    }

2.Cycle:
    {
    - cycle_number : (e.g., 1, 2, ...).
    - discharge_capacity (q_d_n at [cycle_number]).
    - gradient (gradiant_q_d_n at [cycle_number]).
    }

<br>
RELATIONSHIPS:
1.HAS_CYCLE: Links Battery nodes to their respective Cycle nodes.
2.NEXT_CYCLE: Links consecutive Cycle nodes. (add gradient for transitions between cycles here too).

<br>

- (:ChargingProtocol) -[:USED_BY]-> (:Battery) # we will add the ChargingProtocol node later Ignore for now
- (bat_at_t:BatteryID) -[:NEXT_CYCLE]-> ("bat_at_t_plus_one:BatteryID) ; timestep 
- (:Battery) -[:HAS_CYCLE]-> (:BatteryID)


## NEO4J

In [6]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    os.environ["NVIDIA_API_KEY"] = getpass.getpass()

from typing import Any, Dict, List, Optional, Tuple, Type

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import \
    format_to_openai_function_messages
from langchain.agents.output_parsers.openai_tools import \
    OpenAIToolsAgentOutputParser
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun,
                                         CallbackManagerForToolRun)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import AIMessage, HumanMessage
from langchain.tools import BaseTool
from langchain.tools.render import format_tool_to_openai_function

In [7]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

# from langchain_community.graphs import Neo4jGraph
# graph = Neo4jGraph(refresh_schema=False)

# graph.refresh_schema()
# print(graph.schema)


In [12]:
#load the pkl from resources/processed/bat_data_for_kg.pkl
import pickle
import pandas as pd
import numpy as np

path_to_pkl = "/home/jeans/nvaitc/battery_timeseries/resources/processed/bat_data_for_kg.pkl"
with open(path_to_pkl, "rb") as f:
    bat_data = pickle.load(f)
print(bat_data.keys())
print(len(bat_data.keys()))

dict_keys(['b1c1', 'b1c3', 'b1c5', 'b1c7', 'b1c11', 'b1c15', 'b1c17', 'b1c19', 'b1c21', 'b1c24', 'b1c26', 'b1c28', 'b1c30', 'b1c32', 'b1c34', 'b1c36', 'b1c38', 'b1c40', 'b1c42', 'b1c44', 'b2c0', 'b2c2', 'b2c4', 'b2c6', 'b2c11', 'b2c13', 'b2c17', 'b2c19', 'b2c21', 'b2c23', 'b2c25', 'b2c27', 'b2c29', 'b2c31', 'b2c33', 'b2c35', 'b2c37', 'b2c39', 'b2c41', 'b2c43', 'b2c45', 'b1c0', 'b1c2', 'b1c4', 'b1c6', 'b1c9', 'b1c14', 'b1c16', 'b1c18', 'b1c20', 'b1c23', 'b1c25', 'b1c27', 'b1c29', 'b1c31', 'b1c33', 'b1c35', 'b1c37', 'b1c39', 'b1c41', 'b1c43', 'b1c45', 'b2c1', 'b2c3', 'b2c5', 'b2c10', 'b2c12', 'b2c14', 'b2c18', 'b2c20', 'b2c22', 'b2c24', 'b2c26', 'b2c28', 'b2c30', 'b2c32', 'b2c34', 'b2c36', 'b2c38', 'b2c40', 'b2c42', 'b2c44', 'b2c46'])
83


In [ ]:
# MIGHT BE BETTER WAY TO DO THIS as for now we are just using these
# Takse about 2 hours to populate the 35 batter data (out of 83 then error occurs)
import pickle
import os
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

# Neo4j connection details
os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

# Initialize Neo4j connection
neo4j_uri = os.environ["NEO4J_URI"]
neo4j_user = os.environ["NEO4J_USERNAME"]
neo4j_password = os.environ["NEO4J_PASSWORD"]

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

def populate_kg(data):
    with driver.session() as session:
        for battery_id, battery_info in data.items():
            # Create Battery node
            mean_backhalf_grad = battery_info.get('mean_backhalf_grad', None)
            degradation_cycle = battery_info.get('cycle', None)

            session.run(
                """
                MERGE (b:Battery {batteryID: $battery_id})
                SET b.mean_backhalf_grad = $mean_backhalf_grad,
                    b.degradation_cycle = $degradation_cycle
                """,
                battery_id=battery_id,
                mean_backhalf_grad=mean_backhalf_grad,
                degradation_cycle=degradation_cycle,
            )

            # Iterate over cycles
            trimmed_q_d_n = battery_info.get("trimmed_q_d_n", [])
            gradient_q_d_n = battery_info.get("gradient_q_d_n", [])

            previous_cycle_node = None
            for cycle_number, (discharge_capacity, gradient) in enumerate(zip(trimmed_q_d_n, gradient_q_d_n), start=1):
                # Create Cycle node
                cycle_node = session.run(
                    """
                    MERGE (c:Cycle {cycle_number: $cycle_number, batteryID: $battery_id})
                    SET c.discharge_capacity = $discharge_capacity,
                        c.gradient = $gradient
                    RETURN c
                    """,
                    cycle_number=cycle_number,
                    battery_id=battery_id,
                    discharge_capacity=discharge_capacity,
                    gradient=gradient,
                )

                # Create HAS_CYCLE relationship
                session.run(
                    """
                    MATCH (b:Battery {batteryID: $battery_id}), (c:Cycle {cycle_number: $cycle_number, batteryID: $battery_id})
                    MERGE (b)-[:HAS_CYCLE]->(c)
                    """,
                    battery_id=battery_id,
                    cycle_number=cycle_number,
                )

                # Create NEXT_CYCLE relationship
                if previous_cycle_node is not None:
                    session.run(
                        """
                        MATCH (c1:Cycle {cycle_number: $prev_cycle, batteryID: $battery_id}),
                              (c2:Cycle {cycle_number: $current_cycle, batteryID: $battery_id})
                        MERGE (c1)-[:NEXT_CYCLE {gradient: $gradient}]->(c2)
                        """,
                        prev_cycle=cycle_number - 1,
                        current_cycle=cycle_number,
                        battery_id=battery_id,
                        gradient=gradient,
                    )
                previous_cycle_node = cycle_node

# Populate the KG
# populate_kg(bat_data)

print("Knowledge Graph populated successfully.")

[#EBB8]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.126.114.186', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687))): OSError('No data')
Unable to retrieve routing information


ServiceUnavailable: Unable to retrieve routing information

In [ ]:
# def populate_kg_optimized(data):
#     with driver.session() as session:
#         # Begin a transaction
#         with session.begin_transaction() as tx:
#             for battery_id, battery_info in data.items():
#                 # Create Battery node
#                 mean_backhalf_grad = battery_info.get('mean_backhalf_grad', None)
#                 degradation_cycle = battery_info.get('cycle', None)
#                 tx.run(
#                     """
#                     MERGE (b:Battery {batteryID: $battery_id})
#                     SET b.mean_backhalf_grad = $mean_backhalf_grad,
#                         b.degradation_cycle = $degradation_cycle
#                     """,
#                     battery_id=battery_id,
#                     mean_backhalf_grad=mean_backhalf_grad,
#                     degradation_cycle=degradation_cycle,
#                 )

#                 # Prepare Cycles and Relationships
#                 trimmed_q_d_n = battery_info.get("trimmed_q_d_n", [])
#                 gradient_q_d_n = battery_info.get("gradient_q_d_n", [])
                
#                 cycle_queries = []
#                 relationship_queries = []

#                 for cycle_number, (discharge_capacity, gradient) in enumerate(zip(trimmed_q_d_n, gradient_q_d_n), start=1):
#                     # Create Cycle node
#                     cycle_queries.append(
#                         {
#                             "cycle_number": cycle_number,
#                             "battery_id": battery_id,
#                             "discharge_capacity": discharge_capacity,
#                             "gradient": gradient,
#                         }
#                     )
#                     # Create NEXT_CYCLE relationship
#                     if cycle_number > 1:
#                         relationship_queries.append(
#                             {
#                                 "prev_cycle": cycle_number - 1,
#                                 "current_cycle": cycle_number,
#                                 "battery_id": battery_id,
#                                 "gradient": gradient,
#                             }
#                         )

#                 # Execute batch cycle node creation
#                 if cycle_queries:
#                     tx.run(
#                         """
#                         UNWIND $cycles AS cycle
#                         MERGE (c:Cycle {cycle_number: cycle.cycle_number, batteryID: cycle.battery_id})
#                         SET c.discharge_capacity = cycle.discharge_capacity,
#                             c.gradient = cycle.gradient
#                         """,
#                         cycles=cycle_queries,
#                     )

#                 # Create HAS_CYCLE relationships
#                 tx.run(
#                     """
#                     UNWIND $cycles AS cycle
#                     MATCH (b:Battery {batteryID: cycle.battery_id}),
#                           (c:Cycle {cycle_number: cycle.cycle_number, batteryID: cycle.battery_id})
#                     MERGE (b)-[:HAS_CYCLE]->(c)
#                     """,
#                     cycles=cycle_queries,
#                 )

#                 # Create NEXT_CYCLE relationships
#                 if relationship_queries:
#                     tx.run(
#                         """
#                         UNWIND $relationships AS rel
#                         MATCH (c1:Cycle {cycle_number: rel.prev_cycle, batteryID: rel.battery_id}),
#                               (c2:Cycle {cycle_number: rel.current_cycle, batteryID: rel.battery_id})
#                         MERGE (c1)-[:NEXT_CYCLE {gradient: rel.gradient}]->(c2)
#                         """,
#                         relationships=relationship_queries,
#                     )

# # Populate the KG
# populate_kg_optimized(bat_data)

# print("Knowledge Graph populated successfully.")


In [4]:

# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# client = ChatNVIDIA(
#   model="mistralai/mixtral-8x22b-instruct-v0.1",
#   api_key=os.environ["NVIDIA_API_KEY"], 
#   temperature=0.5,
#   top_p=1,
#   max_tokens=1024,
# )
# #test llm
# for chunk in client.stream([{"role":"user","content":"Write a limerick about the wonders of GPU computing."}]): 
#   print(chunk.content, end="")

A GPU so swift and so clever,
In computations it's quite the endeavor.
With its thousands of cores,
On complex tasks it roars,
Solving problems like never, forever!